In [28]:
# type: ignore
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\24746\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\24746\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\24746\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
test_df = pd.read_csv('data/test_nolabel.csv')
train_df = pd.read_csv('data/train.csv')
val_df = pd.read_csv('data/val.csv')

In [30]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)

    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)

    stemmer = PorterStemmer()
    text = ' '.join(stemmer.stem(word) for word in text.split())

    return text

train_df['cleaned_text'] = train_df['text'].apply(clean_text)

In [31]:
emoticon_pattern = re.compile("["  
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # Chinese characters
    u"\U00002702-\U000027B0"
    "]+", flags=re.UNICODE)

train_df = train_df.replace(emoticon_pattern, '', regex=True)

In [32]:
model = Word2Vec(sentences=train_df['cleaned_text'], vector_size=100, window=5, min_count=20, workers=4)
model.save("word2vec.model")

model.build_vocab(train_df['cleaned_text'], progress_per=10000)

In [33]:
word_index = {word: i+1 for i, word in enumerate(model.wv.index_to_key)}

In [34]:
def text_to_sequence(text, word_index):
    return [word_index[word] for word in word_tokenizer(text.lower()) if word in word_index]

In [40]:
model.wv